<a href="https://colab.research.google.com/github/2SEHI/Factory-Anomaly-Analysis/blob/main/2_Display_Sensor_Anomaly_Analysis_VIF%EA%B3%84%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VIF계수를 이용하여 컬럼을 제거하는 처리
- 자동으로 vif계수 구하고 컬럼제거하는 함수를 만들었어요!
잘 돌아가긴 한데 문제는 컬럼을 하나씩  잘 제거하다가 이전의 VIF계수보다 계속 크게 나오는 경우가 생겨서
일정 횟수만큼 이전의 VIF계수보다 크게 나오면 지우는 컬럼의 수를 2개로 늘리는 처리를 추가해야 할 것 같습니다

In [ ]:
import pandas as pd 
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
working_dir = ''
# working_dir = '/content/drive/MyDrive/Colab Notebooks/k-digital/[Project]Display_Sensor_Anomaly_Analysis/'
fact_data = pd.read_csv(working_dir + 'fact_data.csv', index_col='dat...date.name.')

## 1. 모든 컬럼에 대한 VIF계수 구하는 함수

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor


# VIF계수를 구하는 함수
# 매개변수 : 데이터프레임, 컬럼명 리스트
# 반환값 : VIF 데이터프레임, 가장큰 VIF를 가진 컬럼명, 가장 큰 VIF계수
def get_vif(df, columns_all):
#     print('\n',columns_all)
    # 매개변수로 받은 각 피처마다의 VIF 계수를 출력
    vif_df = pd.DataFrame()
    vif_df["VIF Factor"] = [variance_inflation_factor(df[columns_all].values, i) for i in range(df[columns_all].values.shape[1])]
#     print(df[columns_all].columns)
    vif_df["features"] = columns_all
    vif_df.sort_values(by="VIF Factor",ascending=False, inplace=True)
#     print(vif_df)    
    # 가장 큰 vif 컬럼명
    max_col_name = vif_df.iloc[0].features
    # 가장 큰 vif계수
    max_vif = max(vif_df['VIF Factor'])

    return vif_df, max_col_name, max_vif

## 2. 컬럼삭제후 VIF구하는 함수(get_vif호출)
- 이전에 구한 VIF값보다 작아질때까지 실행합니다.

In [ ]:
# VIF 데이터프레임으로 부터 
# 매개변수 : VIF 데이터프레임, 컬럼명 리스트
# 반환값 : VIF 데이터프레임, 
def get_lowest_VIF(df, old_vif_df, old_columns):
    # (삭제한 컬럼명, 컬럼 삭제후 VIF가 가장 큰 컬럼명, VIF계수)
    vif_tp = ()
    # 컬럼 삭제후 VIF
    vif_df = pd.DataFrame()
    # 가장 큰 vif계수
    old_max_vif = max(old_vif_df['VIF Factor'])

    # 삭제한 각 6개의 컬럼에 대해 가장 높은 vif계수를 dict에 저장
    # (삭제한 컬럼명, 컬럼 삭제후 VIF가 가장 큰 컬럼명, VIF계수)
    for i in range(len(old_columns)):
        print('삭제 된 컬럼:', old_vif_df.iloc[i].features)
        old_columns.remove(old_vif_df.iloc[i].features)
        vif_df, max_col_name, max_vif = get_vif(df, old_columns)
        
        print('VIF가 가장 큰 컬럼명, VIF계수 : ', max_col_name, max_vif)
        if old_max_vif > max_vif:
            # (삭제한 컬럼명, 컬럼 삭제후 VIF가 가장 큰 컬럼명, VIF계수)
            vif_tp = (vif_df.iloc[i].features, max_col_name, max_vif)
#             vif_tp_list.append((vif_df.iloc[i].features, max_col_name, max_vif)) 
#              컬럼 삭제후 얻은 vif 데이터프레임 6개 출력
            print(vif_df.head(6),'\n')
            break

    return vif_df, vif_tp

## 3.VIF계수가 30이 될 때까지 컬럼을 제거하는 처리
- get_vif함수와, get_lowest_VIF함수를 호출합니다.

In [ ]:
working_dir = ''
test_df = pd.read_csv(working_dir + 'fact_data_copy.csv', index_col='dat...date.name.')

# 초기 컬럼명 리스트
# 이 컬러명 리스트에서 하나씩 제거해 나갑니다.
cl_columns = list(test_df.columns)

new_vif_df, new_max_col_name, new_max_vif = get_vif(test_df, cl_columns)

while new_max_vif > 30:

    new_vif_df, vif_tp = get_lowest_VIF(test_df, new_vif_df, cl_columns)
    new_max_vif = vif_tp[2]  

# 남겨진 컬럼수 출력
print(len(cl_columns))

삭제 된 컬럼: DCS_Offset_RPM_M11.1WOS45111_PV.1
VIF가 가장 큰 컬럼명, VIF계수 :  DCS_Offset_RPM_M14.1WOS45114_PV.1 5515737449320.877
       VIF Factor                           features
89   5.515737e+12  DCS_Offset_RPM_M14.1WOS45114_PV.1
292  1.626142e+12                      S_AB1_AB2_L_L
264  1.304068e+12                         D_AB1_L_UL
223  8.336140e+11                             bay5_6
88   6.742925e+11                    X1WOS45106_PV.1
284  5.655657e+11                          D_C3_L_UL 

삭제 된 컬럼: DCS_Offset_RPM_M14.1WOS45114_PV.1
VIF가 가장 큰 컬럼명, VIF계수 :  LOWER.C2..Z176.TMP.1TIC42351.PV 2205484636322.476
       VIF Factor                         features
57   2.205485e+12  LOWER.C2..Z176.TMP.1TIC42351.PV
303  1.203849e+12                   S_AB9_AB10_U_L
310  1.036740e+12                      S_C1_C2_L_L
58   8.707656e+11  LOWER.C2..Z182.TMP.1TIC42357.PV
228  7.809936e+11                    l_tin_bay_5_7
291  7.349216e+11                    S_AB1_AB2_L_L 

삭제 된 컬럼: LOWER.C2..Z176.TMP.1TIC

KeyboardInterrupt: 